## Introduction


In this notebook, we use the Bluemix CLI tools to create a new IBM Analytics Engine instance.

*Prerequisites:* 
- You have worked through the notebook `examples/CLI/CLI_Setup.ipynb`

*Recommended:*
- You have worked through the notebook `examples/CLI/Provision_IAE.ipynb`

---

First let's set this notebook to use the full browser width.

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

We can read some variables saved when we ran the notebook `examples/CLI/CLI_Setup.ipynb` to configure our choosen api, org and space

In [ ]:
import json

with open('../../target_endpoint.json') as data_file:    
    target_endpoint = json.load(data_file)
    
api   = target_endpoint['api']
org   = target_endpoint['org']
space = target_endpoint['space']
   
print('api: '   + api)
print('org: '   + org)
print('space: ' + space)

Create a file `../../cos_s3_endpoint.json` with your COS credentials.  The file format should be:

```
{
   "S3_ACCESS_KEY":       "<AccessKey-changeme>",
   "S3_PRIVATE_ENDPOINT": "<Private-EndPoint-changeme>",
   "S3_PUBLIC_ENDPOINT":  "<Public-EndPoint-changeme>",
   "S3_SECRET_KEY":       "<SecretKey-changeme>"
}
```

In [ ]:
with open('../../cos_s3_endpoint.json') as data_file:    
    cos_s3_endpoint = json.load(data_file)
    
S3_ACCESS_KEY       = cos_s3_endpoint['S3_ACCESS_KEY']
S3_PRIVATE_ENDPOINT = cos_s3_endpoint['S3_PRIVATE_ENDPOINT']
S3_PUBLIC_ENDPOINT  = cos_s3_endpoint['S3_PUBLIC_ENDPOINT']
S3_SECRET_KEY       = cos_s3_endpoint['S3_SECRET_KEY']

print('S3 Endpoint: ' + S3_PRIVATE_ENDPOINT)

We can login to Bluemix using the Bluemix CLI

In [ ]:
! bx login --apikey @../../apiKey.json -a {api} -o {org} -s {space}

In [ ]:
custom_script = { 
    "num_compute_nodes": 1, 
    "hardware_config": "Standard", 
    "software_package": "ae-1.0-spark",
    "customization": [{
        "name": "action1",
        "type": "bootstrap",
        "script": {
            "source_type": "http",
            "script_path": "http://raw.githubusercontent.com/snowch/IBM_Analytics_Engine_Examples/master/scripts/COS_S3.sh"
            },
        "script_params": [S3_ACCESS_KEY, S3_PRIVATE_ENDPOINT, S3_SECRET_KEY]
        }]
}

with open('custom_script.json', 'w') as f:
    f.write(json.dumps(custom_script))

We can then attempt to create an IBM Analytics Engine Instance.

In [ ]:
! bx cf create-service IBMAnalyticsEngine Standard 'myiaeinstance' -c custom_script.json

Note the output from above.  If all went ok, the CLI should suggest running `cf service myiaeinstance` to check the provisioning status. 

If there is an error output, jump to the section below on debugging.

Let's do that now.

In [ ]:
! bx cf service myiaeinstance

---

## Upload file to COS

In [ ]:
! pip install boto

In [ ]:
import boto
import boto.s3.connection
import logging

boto.set_stream_logger('paws', level=logging.ERROR)

conn = boto.connect_s3(
        aws_access_key_id = S3_ACCESS_KEY,
        aws_secret_access_key = S3_SECRET_KEY,
        host = S3_PUBLIC_ENDPOINT,
        calling_format = boto.s3.connection.OrdinaryCallingFormat(),
        )

In [ ]:
try:
    bucket = conn.get_bucket('temp-bucket')
except:
    bucket = conn.create_bucket('temp-bucket')
    
print(bucket.name)

In [ ]:
file_contents = """
from __future__ import print_function

import sys
from random import random
from operator import add

from pyspark.sql import SparkSession

if __name__ == "__main__":
    spark = SparkSession.builder.appName("PythonPi").getOrCreate()

    partitions = 2
    n = 100000 * partitions

    def f(_):
        x = random() * 2 - 1
        y = random() * 2 - 1
        return 1 if x ** 2 + y ** 2 <= 1 else 0

    count = spark.sparkContext.parallelize(range(1, n + 1), partitions).map(f).reduce(add)
    print("Pi is roughly %f" % (4.0 * count / n))

    spark.stop()
"""

with open('PiEx.py', 'w') as pyspark_file:
    pyspark_file.write(file_contents)

In [ ]:
key = bucket.new_key('PiEx.py')
key.set_contents_from_filename('PiEx.py')

---

## Analyse data with Spark 

In [ ]:
! bx cf create-service-key myiaeinstance myiaeinstance_servicekey

In [ ]:
! bx cf service-key myiaeinstance myiaeinstance_servicekey > ../../iae_service_key.json

# unfortunately, the output of this command contains some lines of text before the json
# lets remove the first four lines of output and save the raw json

with open('../../iae_service_key.json') as data_file:
    data = ''.join(data_file.readlines()[4:])
    
with open('../../iae_service_key.json', 'w') as data_file:
    data_file.write(data)

In [ ]:
import json

with open('../../iae_service_key.json') as data_file:    
    iae_service_key = json.load(data_file)
    
livy   = iae_service_key['cluster']['service_endpoints']['livy']
user   = iae_service_key['cluster']['user']
pswd   = iae_service_key['cluster']['password']
   
print('livy: '   + livy)
print('user: '   + user)

In [ ]:
import requests

headers = { 
    'Content-Type': 'application/json',
    'X-Requested-By': 'livy'
}
data = { "file":"s3a://{0}/PiEx.py".format(bucket.name) }

res = requests.post(livy, auth=(user, pswd), headers=headers, data=json.dumps(data))

print(res.text)

In [ ]:
headers = { 
    'Content-Type': 'application/json',
    'X-Requested-By': 'livy'
}
res = requests.get(livy, auth=(user, pswd), headers=headers)

res.json()

---

## Debugging 

TODO 

In [ ]:
! bx cf services

In [ ]:
! bx cf delete-service -f 'myiaeinstance'

In [ ]:
!cf delete-service-key -f 'IBM Analytics Engine-pj' 'Credentials-1'